In [1]:
import numpy as np
import os
import glob
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
import collections
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_federated as tff
import random

In [6]:
data_dir_parent = "C:/Users/temmuz/Desktop/2022-2/proje/stanford/dataset"
client_train_dataset = collections.OrderedDict()
client_test_dataset = collections.OrderedDict()
ratio = 0.2
ratio0and5 = 0.2
areas = [[0,2,5],[1,3],[4]]
for client_num in range(3):
    print(client_num)
    train_image_names = []
    test_image_names = []
    labels_train = []
    labels_test = []
    for area in areas[client_num]:
        data_dir = data_dir_parent+"/"+str(area)
        for i in range(11):
            data_dir2 = data_dir+"/"+str(i)+"/*.jpg"
            if len(glob.glob(data_dir2))>0:
                temp_names=glob.glob(data_dir2)
                train, test = train_test_split(temp_names, test_size=0.2, random_state=42)
                test_image_names.extend(test)
                labels_test.extend(np.ones(len(test),dtype=int)*i)
                train_image_names.extend(train)
                labels_train.extend(np.ones(len(train),dtype=int)*i)
    data_train = np.zeros((len(train_image_names),32,32,3))
    data_test = np.zeros((len(test_image_names),32,32,3))
    for i, name in enumerate(train_image_names):
        img1 = PIL.Image.open(name)
        img1 = img1.resize((32,32))
        data_train[i] = tf.keras.preprocessing.image.img_to_array(img1)/255
    for i, name in enumerate(test_image_names):
        img1 = PIL.Image.open(name)
        img1 = img1.resize((32,32))
        data_test[i] = tf.keras.preprocessing.image.img_to_array(img1)/255
    data_train = collections.OrderedDict((('label', labels_train), ('image', data_train)))
    data_test = collections.OrderedDict((('label', labels_test), ('image', data_test)))
    client_train_dataset[client_num] = data_train
    client_test_dataset[client_num] = data_test
train_dataset = tff.simulation.FromTensorSlicesClientData(client_train_dataset)
test_dataset = tff.simulation.FromTensorSlicesClientData(client_test_dataset)

0
1
2


In [7]:
def preprocess(dataset):

    def batch_format_fn(element):
        return collections.OrderedDict(
                x=element['image'],
                y=tf.reshape(element['label'], [-1, 1]))

    return dataset.shuffle(10000, seed=42).batch(128).map(batch_format_fn)

In [8]:
ts = [preprocess(train_dataset.create_tf_dataset_for_client(x)) for x in train_dataset.client_ids]
tes = [preprocess(test_dataset.create_tf_dataset_for_client(x)) for x in test_dataset.client_ids]

In [9]:
class_count_set = np.zeros((3,11))
for client_num in range(3):
    print(client_num)
    ds = ts[client_num]
    for batch in ds:
        for y in batch['y']:
            class_count_set[client_num,y] += 1
    ds = tes[client_num]
    for batch in ds:
        for y in batch['y']:
            class_count_set[client_num,y] += 1

0
1
2


In [13]:
class_count_set = class_count_set.astype(int)

In [14]:
class_count_set

array([[12549,  1119,     0,     0,   924,  7366,   240,   216,   874,
           25,   608],
       [ 7917,  1733,  6371,  2033,     0,  8430,     0,     0,     0,
         1238,  1260],
       [ 8684,   998,     0,   316,     0,  6582,     0,   202,     0,
          514,   297]])